In [1]:
import sys
sys.path.append('../')
from Roma import graphbuilder_v2
from availability_estimation import *

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LO

In [2]:
lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
city = lo_polygon.union(spb_polygon).to_crs(epsg=4326)

russia = ox.geocode_to_gdf("Russia")

In [3]:

citygraph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636,country_polygon=russia)


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 52 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
/Users/polina/Desktop/github/transport_frames/Polina/../Roma/graphbuilder_v2.py:185: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  edges = edges.explode()


In [4]:
import geopandas as gpd

regions = gpd.read_file('russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

In [5]:
carcas = graphbuilder_v2.get_carcas(citygraph,regions,city)
n,e = momepy.nx_to_gdf(carcas)

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_43030/1979102375.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  carcas = graphbuilder_v2.get_carcas(citygraph,regions,city)


In [7]:
n[n['exit']==1 & n['border_region'].isna()].explore()